In [1]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 249, done.
remote: Total 249 (delta 0), reused 0 (delta 0), pack-reused 249
Receiving objects: 100% (249/249), 72.12 MiB | 19.40 MiB/s, done.
Resolving deltas: 100% (123/123), done.


In [1]:
cd first-order-model

/content/first-order-model


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Source画像と動かすための動画を読み込む

In [ ]:
pwd

'/content/first-order-model'

In [37]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('/content/drive/My Drive/FOMM/images/car.jpg')
reader = imageio.get_reader('/content/drive/My Drive/FOMM/04.mp4')


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]

fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

学習済みモデルを読み込む

In [ ]:
pwd

'/content/drive/My Drive/FOMM'

In [38]:
from demo import load_checkpoints

generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/drive/My Drive/FOMM/vox-cpk.pth.tar')

実行

In [39]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 211/211 [00:04<00:00, 45.34it/s]


In [40]:
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 211/211 [00:04<00:00, 46.52it/s]


In [35]:
%cd /content/drive/My Drive/FOMM

/content/drive/My Drive/FOMM


In [ ]:
!ffmpeg -i ./processing/dog.mp4 -ss 00:00:0.00 -t 00:00:12 -filter:v "crop=500:200:150:250" -async 1 dog.mp4


In [ ]:
!ffmpeg -i ./processing/dog.mp4 -ss 00:00:0.00 -t 00:00:12 -filter:v "crop=500:200:500:1000" -async 1 dog2.mp4